# 生成公私钥

In [3]:
from Crypto import Random
from Crypto.PublicKey import RSA
random_generator = Random.new().read  
rsa = RSA.generate(2048, random_generator)  
private_key = rsa.exportKey()
public_key = rsa.publickey().exportKey() 
with open('rsa_private_key.pem', 'wb')as f:
    f.write(private_key)  
with open('rsa_public_key.pem', 'wb')as f:
    f.write(public_key)


# 加密与解密

In [7]:
import base64
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_v1_5 as PKCS1_cipher  


def get_key(key_file):
    with open(key_file) as f:
        data = f.read() 
        key = RSA.importKey(data)
    return key


def encrypt_data(msg): #加密
    public_key = get_key('rsa_public_key.pem')  
    cipher = PKCS1_cipher.new(public_key)  
    encrypt_text = base64.b64encode(cipher.encrypt(msg.encode()))  
    return encrypt_text.decode()  


def decrypt_data(encrypt_msg): #解密
    private_key = get_key('rsa_private_key.pem')  
    cipher = PKCS1_cipher.new(private_key)  
    back_text = cipher.decrypt(base64.b64decode(encrypt_msg), 0)  
    return back_text.decode() 


msg = "Hello World"
encrypt_text = encrypt_data(msg)  # 加密
decrypt_text = decrypt_data(encrypt_text)  # 解密
print('decrypt_text:',decrypt_text,'\n')
print('encrypt_text:',encrypt_text,'\n')

decrypt_text: Hello World 

encrypt_text: D5lIBs/5CHtdRk80n6YIWt9pVbQ1EgN8hHPtHK6MxXnMCXse6rTEXWqV32jK2w0l+d5fSOPnB21sInJvKkkwwKLPlKNQacQUne6ffagNvMUTEYTCWXx6ek9z8oo6aZRpB1WQbj/XEjMmF162yLgFWXNAUtghu1rUqbS/m+y8UHqfzawVUx9Wo/33m/9wtdObnIkrF9x+neIflwsUJP07i5fdR0wrlJ2hVs7VYpHVzUPxunyULoTIBPokBzAWrknFqk8Lz88j12ebKuTrSOx2aM2vfUhb2pqrSSbNGwl4ZI7nr3zr/qFgxDtMgh3Gu3x18+Y6VZnJSv3baulGwsXe9A== 



# 数字签名与验证数字签名

In [8]:
import base64
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5 as PKCS1_signature


def get_key(key_file):
    with open(key_file) as f:
        data = f.read()
        key = RSA.importKey(data)
    return key


def rsa_private_sign(data): #签名
    private_key = get_key('rsa_private_key.pem')  
    signer = PKCS1_signature.new(private_key)  
    digest = SHA.new()  
    digest.update(data.encode())  
    sign = signer.sign(digest)  
    signature = base64.b64encode(sign)  
    signature = signature.decode()  
    return signature  


def rsa_public_check_sign(text, sign): #验签
    publick_key = get_key('rsa_public_key.pem')  
    verifier = PKCS1_signature.new(publick_key)  
    digest = SHA.new()  
    digest.update(text.encode())  
    return verifier.verify(digest, base64.b64decode(sign))  


msg = 'Hello World'
sign = rsa_private_sign(msg)
print(rsa_public_check_sign(msg, sign))

# 返回True验证成功，返回False验证失败

True


# AES加密与解密

In [31]:
from Crypto.Cipher import AES
import base64

# 加密函数
def encrypt(text, key):
    cipher = AES.new(key.encode('utf-8'), AES.MODE_ECB)
    # 补全text长度为AES块大小的倍数
    text = text + (AES.block_size - len(text) % AES.block_size) * chr(AES.block_size - len(text) % AES.block_size)
    encrypted_text = cipher.encrypt(text.encode('utf-8'))
    # 将加密后的字节流转换为base64编码字符串
    encrypted_text_base64 = base64.b64encode(encrypted_text).decode('utf-8')
    return encrypted_text_base64

# 解密函数
def decrypt(encrypted_text_base64, key):
    cipher = AES.new(key.encode('utf-8'), AES.MODE_ECB)
    # 将base64编码字符串转换为加密后的字节流
    encrypted_text = base64.b64decode(encrypted_text_base64.encode('utf-8'))
    decrypted_text = cipher.decrypt(encrypted_text).decode('utf-8')
    # 去除填充的字符
    decrypted_text = decrypted_text[:-ord(decrypted_text[-1])]
    return decrypted_text

if __name__ == '__main__':
    # 使用128位的AES密钥，长度为16字节
    key = 'mysecretpassword'
    # 待加密的明文
    text = 'Hello, World!'
    # 加密
    encrypted_text = encrypt(text, key)
    print('加密后的密文：', encrypted_text)
    # 解密
    decrypted_text = decrypt(encrypted_text, key)
    print('解密后的明文：', decrypted_text)


加密后的密文： uc1bRdYXkE8xoLzvuojaTg==
解密后的明文： Hello, World!
